In [1]:
#!pip install openpyxl

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None

df_gdp = pd.read_excel('data/mpd2023_provisional.xlsx', sheet_name='GDPpc')
df_regions = df_gdp[:2]
df_regions = df_regions.T
df_regions = df_regions[1:]
df_regions = df_regions.reset_index()

df_regions.columns = ['country_name', 'region', 'iso_a3']

df_gfp_values = df_gdp[2:]
df_gfp_values = df_gfp_values[~df_gfp_values['GDP pc 2011 prices'].isna()]
df_gfp_values = df_gfp_values.set_index('GDP pc 2011 prices')
df_gfp_values = df_gfp_values.unstack().reset_index()
df_gfp_values.columns = ['country_name', 'year', 'gdp_pc']
df_gfp_values = df_gfp_values.dropna()
df_gfp_values = df_gfp_values[df_gfp_values['year']<=1900]

df_gfp_values = pd.merge(df_gfp_values, df_regions, on = 'country_name')
df_gfp_values = df_gfp_values.reset_index(drop=True)
df_gfp_values = df_gfp_values.drop(columns=['region'])

len(set(df_gfp_values['country_name']))

77

### Display country GDP per capita

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

show = True

# Create the directory if it doesn't exist
directory = "gdp_pc_country_fig"
if not os.path.exists(directory):
    os.makedirs(directory)
    
# Assuming your DataFrame is named df_gdp
# Group the DataFrame by country_name
grouped = df_gfp_values.groupby('country_name')

# Plot GDP per capita over time for each country
for name, group in grouped:
    plt.scatter(group['year'], group['gdp_pc'], s=10)
    plt.title(name)
    plt.ylabel('Real GDP per capita in 2011$')
    plt.savefig(os.path.join(directory, f'{name}.png'))
    # Show plot
    if show:
        plt.show()

### Load Population

In [4]:
df_population = pd.read_excel('data/mpd2023_provisional.xlsx', sheet_name='Population')
df_population = df_population[2:]
df_population = df_population[~df_population['Population'].isna()]
df_population = df_population.set_index('Population')
df_population = df_population.unstack().reset_index()
df_population.columns = ['country_name', 'year', 'pop']
df_population = df_population[df_population['year']<=1900]

len(set(df_population['country_name']))


169

### Interpolate Population

In [5]:
import itertools

all_years = range(1, 1901)
all_countries = df_population['country_name'].unique()
df_regions_years = pd.DataFrame(list(itertools.product(all_countries, all_years)), columns=['country_name', 'year'])

df_population_all = pd.merge(df_population, df_regions_years, on = ['country_name', 'year'], how = 'outer')
df_population_all = df_population_all.sort_values(by=['country_name', 'year'])

# Bfore 1800, interloate linearly, after 1800, interpolate exponentially
#linear_interpolated = df_population_all[df_population_all['year'] <= 1800]
linear_interpolated = df_population_all.copy()

### Linear Interpolation Before 1800

In [6]:
df_int = []
for country in linear_interpolated['country_name'].unique():
    res= linear_interpolated[linear_interpolated['country_name']==country]
    res['pop'] = pd.to_numeric(res['pop'], errors='coerce')
    res['pop'] = res['pop'].interpolate(how = 'linear')
    df_int.append(res)

linear_interpolated_final = pd.concat([x for x in df_int])
linear_interpolated_final = linear_interpolated_final.dropna().reset_index(drop=True)

In [7]:
test = linear_interpolated_final.copy()
test = test[test['country_name']=='China']
test[test['year']==1100]

,country_name,year,pop
15705,China,1100,67800.0


### Quadratic interpolation after 1800

In [8]:
# exponential_interpolated = df_population_all[df_population_all['year'] >= 1800]

# df_int = []
# for country in exponential_interpolated['country_name'].unique():
#     res= exponential_interpolated[exponential_interpolated['country_name']==country]
#     res['pop'] = pd.to_numeric(res['pop'], errors='coerce')
#     res['pop'] = res['pop'].interpolate(how = 'quadratic')
#     df_int.append(res)

# expo_interpolated_final = pd.concat([x for x in df_int])
# expo_interpolated_final = expo_interpolated_final.dropna().reset_index(drop=True)

### Concat the two datasets

In [9]:
df_population_interpolated = linear_interpolated_final.copy()
# df_population_interpolated = pd.concat([linear_interpolated_final, expo_interpolated_final])
df_population_interpolated = df_population_interpolated.dropna()

### Merge population and GDP per capita

In [10]:
df_gdp_population = pd.merge(df_population_interpolated, df_gfp_values, on = ['country_name', 'year'], how = 'right')

### Regions Analysis

In [11]:
# load regions and equivalent country_name
df_regions_culture = pd.read_csv('../environnement_data/ENS - Cultural Index - Countries Databases - consolidate_table.csv')
df_regions_culture = df_regions_culture[['region', 'iso_a3']].drop_duplicates()
list(set(df_regions_culture.region))
df_regions_culture[df_regions_culture['region']=='Chinese world']

,region,iso_a3
44,Chinese world,MNG
45,Chinese world,TWN
46,Chinese world,CHN


In [12]:
keep_regions_list = ['Italy',
 'Portugal', #OK
 'Low countries', # Weighted Average of Belgium and Netherlands normalized by population
 'Nordic countries', # Average of Norway, Sweden, Denmark and Iceland normalized by population
 'Indian world',
 'Chinese world',
 'Korea',
 'Arabic world',
 'Greek World',
 'Eastern Europe',
 'France',
 'Persian world',
 'Japan',
 'Latin World',
 'Spain',
 'United Kingdom',
 'Balkans',
 'German world']

In [21]:
df_gdp_pop_regions = pd.merge(df_gdp_population, df_regions_culture, on = ['iso_a3'])
df_gdp_pop_regions = df_gdp_pop_regions[df_gdp_pop_regions['region'].isin(keep_regions_list)]

df_gdp_weighted = df_gdp_pop_regions.groupby(['region', 'year']).apply(lambda x: (x['gdp_pc'] * x['pop']).sum() / x['pop'].sum())
df_gdp_weighted = df_gdp_weighted.reset_index()
df_gdp_weighted = df_gdp_weighted.rename(columns={0: 'gdp_pc_weighted'})

df_gdp_weighted = df_gdp_weighted[df_gdp_weighted['year']>1]

### Display regions GDP per capita

In [29]:
df_gdp_weighted['decade'] = df_gdp_weighted['year'] // 10 * 10
df_gdp_weighted = df_gdp_weighted.drop_duplicates(subset=['region', 'decade'], keep='first')
df_gdp_weighted = df_gdp_weighted[df_gdp_weighted['year']>=1000]
df_gdp_weighted = df_gdp_weighted[df_gdp_weighted['year']<=1880]
df_gdp_weighted = df_gdp_weighted.reset_index(drop=True)

,region,year,gdp_pc_weighted,decade
0,Arabic world,1000,1056.285714,1000
1,Arabic world,1120,1060.638037,1120
2,Arabic world,1280,1068.000000,1280
3,Arabic world,1300,972.000000,1300
4,Arabic world,1400,1164.000000,1400
...,...,...,...,...
657,United Kingdom,1840,4018.000000,1840
658,United Kingdom,1850,4332.000000,1850
659,United Kingdom,1860,5086.000000,1860
660,United Kingdom,1870,5829.000000,1870


### Add Population

In [37]:
df_pop_regions = df_gdp_pop_regions.groupby(['region', 'year']).apply(lambda x: x['pop'].sum()).reset_index()
df_pop_regions = df_pop_regions.rename(columns={0: 'pop'})

### Wieghted GDP and Population

In [40]:
df_gdp_weighted_pop = pd.merge(df_gdp_weighted, df_pop_regions, on = ['region', 'year'])
df_gdp_weighted_pop.to_csv('results/gdp_wieghted_pop.csv', index=False)

In [41]:
df_gdp_weighted_pop.head(10)

,region,year,gdp_pc_weighted,decade,pop
0,Arabic world,1000,1056.285714,1000,7000.0
1,Arabic world,1120,1060.638037,1120,6520.0
2,Arabic world,1280,1068.000000,1280,4440.0
3,Arabic world,1300,972.000000,1300,4400.0
4,Arabic world,1400,1164.000000,1400,4200.0
5,Arabic world,1500,1084.000000,1500,4000.0
6,Arabic world,1820,832.465167,1820,15517.0
7,Arabic world,1870,1091.754910,1870,22453.0
8,Balkans,1833,1537.000000,1830,2605.0
9,Balkans,1840,1932.000000,1840,2777.0


## Display

In [ ]:
show = True

# Create the directory if it doesn't exist
directory = "gdp_pc_region_fig"
if not os.path.exists(directory):
    os.makedirs(directory)
    
# Assuming your DataFrame is named df_gdp
# Group the DataFrame by country_name
grouped = df_gdp_weighted.groupby('region')

# Plot GDP per capita over time for each country
for name, group in grouped:
    plt.scatter(group['year'], group['gdp_pc_weighted'], s=10)
    plt.title(name)
    plt.ylabel('Real GDP per capita in 2011$ (Population Weighted Average)')
    plt.savefig(os.path.join(directory, f'{name}.png'))
    # Show plot
    if show:
        plt.show()

<!-- ### Load Culture -->

In [39]:
# df_culture = pd.read_csv('../unseen_species_model/results/occupation/unseen_data_occupation.csv', index_col=[0])
# df_culture = df_culture.drop(columns=['occupation', 'model_type'])
# df_culture = df_culture.groupby(['region_name', 'decade']).sum().reset_index()